In [119]:
from scraper.scraper import Scraper 
bot = Scraper()
bot.accept_cookies()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/FKhan/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


In [120]:
bot.keys_search('Mens adidas t-shirts')

In [27]:
bot.next_button()

<selenium.webdriver.remote.webelement.WebElement (session="ed54465c95759096bde24fad89e7c477", element="37594e66-512d-4ea5-8bb9-de34473dc76b")>

In [121]:
container = bot.product_container()

In [123]:
from lib2to3.pgen2 import driver
from time import time
from typing_extensions import Self
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager


In [124]:
list_properties = container.find_elements(By.XPATH, './ul/li')
imageLinks = container.find_elements(By.XPATH, "./ul/li/span/a/picture")

In [125]:
link_list = []
for property in list_properties:
    link_list.append(property.find_element(By.TAG_NAME, 'a').get_attribute('href'))
imageNames = []
for element in imageLinks:
    imageNames.append(element.find_element(By.TAG_NAME, 'img').get_attribute("src"))    

In [63]:
from sqlalchemy import create_engine
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'dcp1.ckiqoi4xq1wx.eu-west-2.rds.amazonaws.com' 
USER = 'postgres'
PASSWORD = 'Hamburg98'
PORT = 5432
DATABASE = 'postgres'
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")
df = pd.read_sql('test_1', engine)
id_scraper = list(df['id'])

In [126]:
import time
import uuid
property_dict = {'link':[],
                 'price':[],
                 'name':[],
                 'id' :[],
                 'UUID':[],
                 'imagelinks_1':[],
                }


for link in link_list[0:73]:
    # bot.driver.get(link)
    # time.sleep(1)
    # id = bot.driver.find_element(By.XPATH, '//span[@class="product-code"]')
    # property_dict['id'].append(id.text)
    # if id in id_scraper:
    #     continue
    # else:
        bot.driver.get(link)
        bot.driver.execute_script("window.scrollTo(0,1100)")  
        time.sleep(4)
        bot.clicker()
        time.sleep(1)
        property_dict['link'].append(link)
        price = bot.driver.find_element(By.XPATH, '//span[@data-e2e="product-price"]')
        property_dict['price'].append(price.text)
        name = bot.driver.find_element(By.XPATH, '//h1[@itemprop="name"]')
        property_dict['name'].append(name.text)
        id = bot.driver.find_element(By.XPATH, '//span[@class="product-code"]')
        property_dict['id'].append(id.text)   
        property_dict['UUID'].append(uuid.uuid4())
        # imageLinks_1 = WebDriverWait(bot.driver, 6).until(EC.presence_of_element_located((By.TAG_NAME, 'img').get_attribute("src")))
        # # bot.driver.find_element(By.TAG_NAME, 'img').get_attribute("src")
        # property_dict['imagelinks_1'].append(imageLinks_1)

for image in imageNames[0:73]:
    bot.driver.get(image)
    time.sleep(1)
    property_dict['imagelinks_1'].append(image)        


no filted added
no filted added
no filted added
no filted added
no filted added
no filted added
no filted added
no filted added
no filted added
no filted added
no filted added
no filted added
no filted added
no filted added
no filted added
no filted added
no filted added
no filted added
no filted added
no filted added
no filted added


In [108]:
for image in imageNames[0:7]:
    bot.driver.get(image)
    time.sleep(1)
    property_dict['imagelinks_1'].append(image)   

In [127]:
property_dict

{'link': ['https://www.jdsports.co.uk/product/black-adidas-originals-adi-edge-t-shirt/16480643/',
  'https://www.jdsports.co.uk/product/orange-adidas-originals-california-3-stripes-t-shirt/16265513/',
  'https://www.jdsports.co.uk/product/white-adidas-originals-retro-back-graphic-t-shirt/16481543/',
  'https://www.jdsports.co.uk/product/black-adidas-originals-itasca-22-t-shirt/16476586/',
  'https://www.jdsports.co.uk/product/black-adidas-originals-retro-t-shirt/16481140/',
  'https://www.jdsports.co.uk/product/brown-adidas-originals-california-3-stripes-t-shirt/16474480/',
  'https://www.jdsports.co.uk/product/blue-adidas-originals-california-3-stripes-t-shirt/16471879/',
  'https://www.jdsports.co.uk/product/yellow-adidas-originals-california-3-stripes-t-shirt/16471084/',
  'https://www.jdsports.co.uk/product/blue-adidas-originals-itasca-22-t-shirt/16471994/',
  'https://www.jdsports.co.uk/product/grey-adidas-originals-california-3-stripes-t-shirt/16471878/',
  'https://www.jdsports.

In [130]:
import json
from uuid import UUID


class UUIDEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, UUID):
            return obj.hex
        return json.JSONEncoder.default(self, obj)
j = json.dumps(property_dict, cls=UUIDEncoder, indent=4)
f = open('Adidas_t-shirts.json', 'w')
print(j, file=f)
f.close()

In [128]:
import pandas as pd 
x = pd.DataFrame(property_dict)

In [129]:
x

,link,price,name,id,UUID,imagelinks_1
0,https://www.jdsports.co.uk/product/black-adida...,£30.00,adidas Originals Adi Edge T-Shirt,Product Code: 16480643/515177,bfb1a0c4-39ff-4054-b841-0b38e9814bc9,https://i8.amplience.net/t/jpl/jd_product_list...
1,https://www.jdsports.co.uk/product/orange-adid...,£28.00,adidas Originals California 3-Stripes T-Shirt,Product Code: 16265513/509777,6fd3ad9b-e7cb-4b37-8cf7-a9c450406be3,https://i8.amplience.net/t/jpl/jd_product_list...
2,https://www.jdsports.co.uk/product/white-adida...,£33.00,adidas Originals Retro Back Graphic T-Shirt,Product Code: 16481543/557662,1b6ee491-ca9d-4e2f-8e09-2649ee7770ea,https://i8.amplience.net/t/jpl/jd_product_list...
3,https://www.jdsports.co.uk/product/black-adida...,£30.00,adidas Originals Itasca 22 T-Shirt,Product Code: 16476586/508582,ac0a116f-5730-4860-87c8-05204b6844c4,https://i8.amplience.net/t/jpl/jd_product_list...
4,https://www.jdsports.co.uk/product/black-adida...,£33.00,adidas Originals Retro T-Shirt,Product Code: 16481140/557660,2c48cdc1-c3ab-42dd-ba88-b00109a41504,https://i8.amplience.net/t/jpl/jd_product_list...
...,...,...,...,...,...,...
67,https://www.jdsports.co.uk/product/adidas-orig...,£28.00,adidas Originals Graphics Behind the Trefoil T...,Product Code: 16241769,f5ab3c73-4418-4c49-ac2f-66d7eb021d47,https://i8.amplience.net/t/jpl/jd_product_list...
68,https://www.jdsports.co.uk/product/adidas-orig...,£28.00,adidas Originals Yung Z T-Shirt,Product Code: 16241699/501727,ec95c30b-1778-4336-bf52-34ef364aaf1e,https://i8.amplience.net/t/jpl/jd_product_list...
69,https://www.jdsports.co.uk/product/grey-adidas...,£33.00,adidas Originals Adventure C Butterfly Pocket ...,Product Code: 16240380/501751,bd93efea-c0bd-4e77-ac0b-908239de69e2,https://i8.amplience.net/t/jpl/jd_product_list...
70,https://www.jdsports.co.uk/product/adidas-orig...,£28.00,adidas Originals SPRT Outline Logo T-Shirt,Product Code: 16240343,fb866bb2-f79e-45c1-b895-fbd9292d21ec,https://i8.amplience.net/t/jpl/jd_product_list...


In [7]:
import pandas as pd
lf = pd.read_json('/Users/FKhan/Downloads/miniconda3/DCP_Project/raw_data/Nike caps/data.json')
y = pd.DataFrame(lf)

In [113]:
from sqlalchemy import create_engine
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'dcp1.ckiqoi4xq1wx.eu-west-2.rds.amazonaws.com' 
USER = 'postgres'
PASSWORD = 'Hamburg98'
PORT = 5432
DATABASE = 'postgres'
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")
df = pd.read_sql('test_1', engine)
id_scraper = list(df['id'])

In [131]:
x.to_sql('Adidas_t-shirts', con= engine, if_exists='append', index=False )

72

In [98]:
imageLinks = container.find_elements(By.XPATH, "./ul/li/span/a/picture")

In [99]:
imageNames = []
for element in imageLinks:
    imageNames.append(element.find_element(By.TAG_NAME, 'img').get_attribute("src"))

In [132]:
bot.multiple_image_2(list=imageNames[0:73], file_path='/Users/FKhan/Downloads/miniconda3/DCP_Project/raw_data/Adidas_t-shirts/')

In [396]:
# this function will aid with downloading images using the specific image link (its another way of dowloading compared to the one below)
import urllib.request
def search_image():
        url = 'https://i8.amplience.net/t/jpl/jd_product_list?plu=jd_355655_plc&qlt=92&w=363&h=463&v=1&fmt=auto'
        opener = urllib.request.build_opener()
        opener.addheaders = [('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36')]
        urllib.request.install_opener(opener)
        urllib.request.urlretrieve(
        url, f'Nike_Hat_27.png')
        print(f'downloaded!')
try:
    search_image()
except: print('Thats not a valid url')

downloaded!


In [5]:
# This function saves images using the product link rather than image link and saves in chosen path
bot.image_download(url = 'https://www.jdsports.co.uk/product/adidas-originals-camo-baseball-cap/16239652/', file_name='adidas_cap_2', file_path='/Users/FKhan/Downloads/miniconda3/DCP_Project/raw_data/')

In [116]:
import boto3
s3_client = boto3.client('s3')

In [117]:
# To upload multple files
import glob
png_files = glob.glob('/Users/FKhan/Downloads/miniconda3/DCP_Project/raw_data/Adidas_FTS/*.png')

for filename in png_files:
    s3_client.upload_file(filename, 'dcp1', filename)

In [118]:
# To upload individual files
response = s3_client.upload_file('/Users/FKhan/Downloads/miniconda3/DCP_Project/raw_data/Adidas_FTS/Adidas_FTS.json', 'dcp1', 'Adidas_FTS.json')

In [49]:
# To download files from a bucket
import requests
url = 'https://dcp1.s3.eu-west-2.amazonaws.com/Nike+caps/Nike_Hat_023018_023018.png'

response = requests.get(url)
with open('blastoise.png', 'wb') as f:
    f.write(response.content)